In [ ]:
import os,sys
import kfp

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(token):
    train       = dkube_training_op(token, '{"image":"ocdr/d3-datascience-pytorch-cpu:v1.6"}',
                                    framework="pytorch", version="1.6",
                                    program="pytorch-examples", run_script="python mnist/train.py",
                                    datasets='["mnist"]', outputs='["mnist"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]')

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/pytorchserver:1.6"}',
                                    transformer_image='{"image":"ocdr/d3-datascience-pytorch-cpu:v1.6-1"}',
                                    transformer_project="pytorch-examples",
                                    transformer_code='mnist/transformer.py').after(train)

In [ ]:
client.create_run_from_pipeline_func(mnist_pipeline, arguments={"token":token})

In [ ]:
#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline.zip")
client.upload_pipeline("mnist-pipeline.zip")